# Instalação de Dependências 

In [ ]:
%pip install pandas

: 

In [ ]:
%pip install pyarrow fastparquet

: 

In [4]:
import pandas as pd

# Tratamento inical dos dados do arquivo 'FALHAS_PREDICT.csv'

In [ ]:
df_falhas = pd.read_excel('FALHAS_PREDICT.xlsx')
df_falhas

In [ ]:
df_falhas.reset_index(drop=True, inplace=True)

df_falhas.head()

In [ ]:
df_falhas

## Ajustando a linha correta como cabeçalho

In [ ]:
# Definir a linha correta como cabeçalho
df_falhas.columns = df_falhas.iloc[1]
df_falhas = df_falhas[2:]

# Resetar o índice do DataFrame
df_falhas.reset_index(drop=True, inplace=True)

df_falhas.head()

In [ ]:
df_falhas.columns

## Removendo coluna NaN

In [ ]:
# Remover a coluna `NaN`
df_falhas.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_falhas.head()

## Removendo Duplicatas

In [ ]:
df_falhas = df_falhas.drop_duplicates()
df_falhas.info()

In [ ]:
df_falhas.head()

## Retirando colunas que não serão utilizadas nessa versão do modelo

In [14]:
# Dropar colunas
df_falhas.drop(['MODELO', 'COR', 'MOTOR', 'ESTACAO', 'USUARIO'], axis=1, inplace=True)

In [ ]:
df_falhas.head()

## Filtrando ZPs

In [16]:
# Filtrando para manter apenas valores entre ZP1 e ZP7
df_falhas = df_falhas[df_falhas['HALLE'].str.match(r'ZP[1-7]', na=False)]

In [ ]:
df_falhas.head()

## Gerando um novo arquivo do dataset utilizando parquet

In [18]:
df_falhas.to_parquet('df_falhas.parquet', index=False)

In [ ]:
df_falhas = pd.read_parquet('df_falhas.parquet')
df_falhas.head()

# Tratamento e Ajustes nos dados de resultado

In [20]:
import pandas as pd

## RESULTADOS 02

## Nas células a seguir foram selecionadas as colunas necessarias para o desenvolvimento desse modelo e o tratamento para que não existam dados duplicados. Por fim, foi gerado um novo dataset utilizando parquet.

In [ ]:
df_result02 = pd.read_excel('RESULTADOS_02_03_2024.xlsx')

In [ ]:
df_result02.columns

In [ ]:
df_result02.columns

In [ ]:

df_result02.head()

In [25]:
df_result02.columns = df_result02.iloc[0]
df_result02 = df_result02[1:]

In [26]:

df_result02.columns = df_result02.columns.astype(str)

In [27]:
df_result02 = df_result02.drop_duplicates()

In [ ]:
df_result02.head()

In [ ]:
# Identificar todas as duplicatas
duplicates = df_result02[df_result02.duplicated(keep=False)]

print(duplicates)

In [ ]:
df_result02 = df_result02.drop_duplicates()
df_result02.info()

In [ ]:
df_result02

In [32]:
df_result02.to_parquet('df_result02.parquet', index=False)

In [ ]:
df_result02 = pd.read_parquet('df_result02.parquet')
df_result02

In [ ]:
df_result02 = df_result02.loc[:, df_result02.columns != 'nan']
df_result02 = df_result02.loc[:, df_result02.columns != 'ID']
df_result02 = df_result02.loc[:, df_result02.columns != 'STATUS']
df_result02

#remover colunas inuteis

In [ ]:
num_unicos = df_result02['NAME'].nunique()
print(f"A coluna 'NAME' possui {num_unicos} valores únicos.")

In [ ]:
valores_unicos = df_result02['NAME'].unique()
print("Valores únicos na coluna 'NAME':")
print(valores_unicos)

In [ ]:
frequencia = df_result02['NAME'].value_counts()
print("Frequência de cada valor único na coluna 'NAME':")
print(frequencia)

## Explorando Value_id e Name


In [38]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import f_oneway


In [ ]:
# Agrupar por 'NAME' e calcular estatísticas descritivas de 'VALUE_ID'
estatisticas = df_result02.groupby('NAME')['VALUE_ID'].describe()
print(estatisticas)


In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(x='NAME', y='VALUE_ID', data=df_result02)
plt.title('Distribuição de VALUE_ID por NAME')
plt.xticks(rotation=45)
plt.show()

In [ ]:
medias = df_result02.groupby('NAME')['VALUE_ID'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='NAME', y='VALUE_ID', data=medias)
plt.title('Média de VALUE_ID por NAME')
plt.xticks(rotation=45)
plt.show()



In [ ]:
# Criar lista de valores de 'VALUE_ID' para cada grupo de 'NAME'
grupos = [grupo['VALUE_ID'].dropna().values for nome, grupo in df_result02.groupby('NAME')]

# Realizar o teste ANOVA
estatistica, p_valor = f_oneway(*grupos)
print(f'Estatística F: {estatistica}')
print(f'Valor-p: {p_valor}')

### Explorando Unity, Value e ValueID

In [ ]:
# Verificar os tipos de dados
print(df_result02[['UNIT', 'VALUE', 'VALUE_ID']].dtypes)

# Converter 'VALUE' e 'VALUE_ID' para numérico, se necessário
df_result02['VALUE'] = pd.to_numeric(df_result02['VALUE'], errors='coerce')
df_result02['VALUE_ID'] = pd.to_numeric(df_result02['VALUE_ID'], errors='coerce')

# Remover valores nulos nas colunas de interesse
df_result02 = df_result02.dropna(subset=['UNIT', 'VALUE', 'VALUE_ID'])


In [ ]:
# Estatísticas descritivas de 'VALUE'
print("Estatísticas de 'VALUE':")
print(df_result02['VALUE'].describe())

# Estatísticas descritivas de 'VALUE_ID'
print("\nEstatísticas de 'VALUE_ID':")
print(df_result02['VALUE_ID'].describe())

# Valores únicos em 'UNIT'
print("\nValores únicos em 'UNIT':")
print(df_result02['UNIT'].unique())


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(df_result02['VALUE_ID'], df_result02['VALUE'], alpha=0.5)
plt.title('Relação entre VALUE_ID e VALUE')
plt.xlabel('VALUE_ID')
plt.ylabel('VALUE')
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
df_result02.boxplot(column='VALUE', by='UNIT', grid=False)
plt.title('Distribuição de VALUE por UNIT')
plt.suptitle('')
plt.xlabel('UNIT')
plt.ylabel('VALUE')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Histograma de VALUE
plt.figure(figsize=(8, 6))
plt.hist(df_result02['VALUE'], bins=30, alpha=0.7)
plt.title('Distribuição de VALUE')
plt.xlabel('VALUE')
plt.ylabel('Frequência')
plt.show()

# Histograma de VALUE_ID
plt.figure(figsize=(8, 6))
plt.hist(df_result02['VALUE_ID'], bins=30, alpha=0.7, color='orange')
plt.title('Distribuição de VALUE_ID')
plt.xlabel('VALUE_ID')
plt.ylabel('Frequência')
plt.show()


In [ ]:
import seaborn as sns

# Matriz de correlação
corr = df_result02[['VALUE', 'VALUE_ID']].corr()
print("Matriz de correlação:")
print(corr)

# Heatmap da correlação
plt.figure(figsize=(6, 4))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlação entre VALUE e VALUE_ID')
plt.show()


In [ ]:
agrupamento = df_result02.groupby('UNIT')[['VALUE', 'VALUE_ID']].mean()
print("\nMédias de VALUE e VALUE_ID por UNIT:")
print(agrupamento)


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='VALUE_ID', y='VALUE', hue='UNIT', data=df_result02)
plt.title('Relação entre VALUE_ID e VALUE colorido por UNIT')
plt.xlabel('VALUE_ID')
plt.ylabel('VALUE')
plt.legend(title='UNIT', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


In [ ]:
# Converter 'VALUE' e 'VALUE_ID' para numérico
df_result02['VALUE'] = pd.to_numeric(df_result02['VALUE'], errors='coerce')
df_result02['VALUE_ID'] = pd.to_numeric(df_result02['VALUE_ID'], errors='coerce')


In [55]:
# Remover linhas com valores nulos em 'VALUE' ou 'VALUE_ID'
df_result02 = df_result02.dropna(subset=['VALUE', 'VALUE_ID'])


In [ ]:
num_unique_value = df_result02['VALUE'].nunique()
num_unique_value_id = df_result02['VALUE_ID'].nunique()

print(f"Número de valores únicos em 'VALUE': {num_unique_value}")
print(f"Número de valores únicos em 'VALUE_ID': {num_unique_value_id}")


In [57]:
# Ajustar 'q' com base no número de valores únicos
q_value = min(4, num_unique_value)
q_value_id = min(4, num_unique_value_id)

# Categorizar 'VALUE'
df_result02['VALUE_cat'] = pd.qcut(df_result02['VALUE'], q=q_value, labels=False, duplicates='drop')

# Categorizar 'VALUE_ID'
df_result02['VALUE_ID_cat'] = pd.qcut(df_result02['VALUE_ID'], q=q_value_id, labels=False, duplicates='drop')


In [ ]:
print(df_result02['VALUE_cat'].isnull().sum())
print(df_result02['VALUE_ID_cat'].isnull().sum())


In [ ]:
from sklearn.metrics import confusion_matrix

# Criar a matriz de confusão
conf_mat = confusion_matrix(df_result02['VALUE_ID_cat'], df_result02['VALUE_cat'])

# Exibir a matriz de confusão
print("\nMatriz de Confusão entre VALUE_ID_cat e VALUE_cat:")
print(conf_mat)

# Visualizar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Q1', 'Q2', 'Q3', 'Q4'],
            yticklabels=['Q1', 'Q2', 'Q3', 'Q4'])
plt.xlabel('VALUE_cat')
plt.ylabel('VALUE_ID_cat')
plt.title('Matriz de Confusão entre VALUE_ID_cat e VALUE_cat')
plt.show()


In [ ]:
# Codificar 'UNIT' numericamente
df_result02['UNIT_code'] = df_result02['UNIT'].astype('category').cat.codes

conf_mat_unit = confusion_matrix(df_result02['UNIT_code'], df_result02['VALUE_cat'])

unit_labels = df_result02['UNIT'].astype('category').cat.categories

plt.figure(figsize=(12, 8))
sns.heatmap(conf_mat_unit, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Q1', 'Q2', 'Q3', 'Q4'],
            yticklabels=unit_labels)
plt.xlabel('VALUE_cat')
plt.ylabel('UNIT')
plt.title('Matriz de Confusão entre UNIT e VALUE_cat')
plt.show()


# Exploração dos Dados


In [ ]:
df_result02 = pd.read_excel('RESULTADOS_02_03_2024.xlsx')



In [ ]:
df_result02.columns
df_result02.head()


In [54]:
df_result02.columns = df_result02.iloc[0]
df_result02 = df_result02[1:]
df_result02.columns = df_result02.columns.astype(str)
df_result02 = df_result02.drop_duplicates()

In [55]:
df_result02 = df_result02.loc[:, df_result02.columns != 'nan']

In [ ]:
df_result02.head()

In [ ]:

unique_names = df_result02['NAME'].unique()

print("Valores únicos na coluna 'NAME':")
print(unique_names)



In [ ]:
num_unique_names = df_result02['NAME'].nunique()
print(f"Número de valores únicos na coluna 'NAME': {num_unique_names}")


In [ ]:
unique_value_ids = df_result02['VALUE_ID'].unique()

print("Valores únicos na coluna 'VALUE_ID':")
print(unique_value_ids)

In [ ]:
num_unique_value_ids = df_result02['VALUE_ID'].nunique()
print(f"Número de valores únicos na coluna 'VALUE_ID': {num_unique_value_ids}")

In [ ]:
num_null_names = df_result02['NAME'].isnull().sum()
print(f"Número de valores nulos na coluna 'NAME': {num_null_names}")


num_null_value_ids = df_result02['VALUE_ID'].isnull().sum()
print(f"Número de valores nulos na coluna 'VALUE_ID': {num_null_value_ids}")


In [62]:
df_result02 = df_result02.dropna(subset=['VALUE_ID'])

In [ ]:
unique_names_sorted = sorted(unique_names)
unique_value_ids_sorted = sorted(unique_value_ids)

print("Valores únicos na coluna 'NAME' (ordenados):")
print(unique_names_sorted)

print("Valores únicos na coluna 'VALUE_ID' (ordenados):")
print(unique_value_ids_sorted)

In [ ]:
# Agrupar por 'NAME' e 'VALUE_ID' e contar as ocorrências
contagens_name_valueid = df_result02.groupby(['NAME', 'VALUE_ID']).size().reset_index(name='counts')

# Exibir as contagens
print(contagens_name_valueid)


In [ ]:
# Filtrar as combinações onde counts > 1
combinacoes_repetidas = contagens_name_valueid[contagens_name_valueid['counts'] > 1]

# Exibir as combinações repetidas
print("Combinações repetidas de 'NAME' e 'VALUE_ID':")
print(combinacoes_repetidas)


In [ ]:
# Ordenar as combinações por 'counts' em ordem decrescente
contagens_ordenadas = contagens_name_valueid.sort_values(by='counts', ascending=False)

print("Todas as combinações de 'NAME' e 'VALUE_ID' ordenadas por contagem:")
print(contagens_ordenadas)


In [ ]:
# Criar a tabela dinâmica
tabela_pivot = contagens_name_valueid.pivot(index='NAME', columns='VALUE_ID', values='counts').fillna(0)

# Exibir a tabela
print(tabela_pivot)


In [ ]:
# Contar o número de 'VALUE_ID's únicos para cada 'NAME'
contagem_valueid_por_name = df_result02.groupby('NAME')['VALUE_ID'].nunique().reset_index(name='unique_valueid_count')

# Filtrar os 'NAME's que têm mais de um 'VALUE_ID' único
names_multiplos_valueid = contagem_valueid_por_name[contagem_valueid_por_name['unique_valueid_count'] > 1]

print("Nomes associados a múltiplos 'VALUE_ID's:")
print(names_multiplos_valueid)


In [ ]:
# Contar o número de 'NAME's únicos para cada 'VALUE_ID'
contagem_name_por_valueid = df_result02.groupby('VALUE_ID')['NAME'].nunique().reset_index(name='unique_name_count')

# Filtrar os 'VALUE_ID's que têm mais de um 'NAME' único
valueid_multiplos_names = contagem_name_por_valueid[contagem_name_por_valueid['unique_name_count'] > 1]

print("VALUE_ID's associados a múltiplos 'NAME's:")
print(valueid_multiplos_names)


In [74]:
# Agrupar por 'NAME' e 'VALUE_ID' e contar as ocorrências
contagens_name_valueid = df_result02.groupby(['NAME', 'VALUE_ID']).size().reset_index(name='Counts')


In [ ]:
# Ordenar a tabela por 'Counts' em ordem decrescente
contagens_name_valueid_sorted = contagens_name_valueid.sort_values(by='Counts', ascending=False)

# Exibir a tabela ordenada
print("Tabela Ordenada de Contagens para 'NAME' e 'VALUE_ID':")
print(contagens_name_valueid_sorted)


In [77]:
# Criar a tabela dinâmica
pivot_table = contagens_name_valueid.pivot(index='NAME', columns='VALUE_ID', values='Counts').fillna(0)


In [80]:
from sklearn.preprocessing import LabelEncoder

# Inicializar os codificadores de labels
le_name = LabelEncoder()
le_value_id = LabelEncoder()

# Ajustar e transformar as colunas
df_result02['NAME_encoded'] = le_name.fit_transform(df_result02['NAME'])
df_result02['VALUE_ID_encoded'] = le_value_id.fit_transform(df_result02['VALUE_ID'])


In [81]:
from sklearn.metrics import confusion_matrix

# Calcular a matriz de confusão
conf_matrix = confusion_matrix(df_result02['NAME_encoded'], df_result02['VALUE_ID_encoded'])


In [83]:
# Selecionar os top N 'NAME's mais frequentes
top_n_names = df_result02['NAME'].value_counts().nlargest(10).index
df_top = df_result02[df_result02['NAME'].isin(top_n_names)]

# Repetir o processo de codificação e plotagem com df_top


## RESULTADOS 04

## Nas células a seguir foram selecionadas as colunas necessarias para o desenvolvimento desse modelo e o tratamento para que não existam dados duplicados. Por fim, foi gerado um novo dataset utilizando parquet.

In [ ]:
df_result04 = pd.read_csv('RESULTADOS_04_06_2024_full_teste.csv', low_memory=False, compression='gzip')
df_result04.head()

In [ ]:
# Dropar uma coluna e modificar o DataFrame existente
df_result04.drop(['Unnamed: 0', 'NAME', 'STATUS', 'UNIT', 'VALUE_ID', 'VALUE'], axis=1, inplace=True)
df_result04.head()

In [ ]:
df_result04 = df_result04.drop_duplicates()
df_result04

In [ ]:
df_result04.to_parquet('df_result04.parquet', index=False)
df_result04 = pd.read_parquet('df_result04.parquet')
df_result04

: 

# Extração de Features 
## Name e Value_ID

In [ ]:
# Exibir os primeiros valores de 'NAME'
print(df_result02['NAME'].head())


: 

In [ ]:
# Dividir 'NAME' em partes
df_result02[['PART1', 'PART2', 'PART3']] = df_result02['NAME'].str.split('_', expand=True)

# Exibir as novas colunas
print(df_result02[['NAME', 'PART1', 'PART2', 'PART3']].head())


In [ ]:
# Converter 'PART3' para numérico
df_result02['PART3_num'] = pd.to_numeric(df_result02['PART3'], errors='coerce')

# Exibir a nova coluna
print(df_result02[['PART3', 'PART3_num']].head())


In [ ]:
# One-Hot Encoding para 'PART1' e 'PART2'
df_encoded = pd.get_dummies(df_result02, columns=['PART1', 'PART2'])

# Exibir as novas colunas
print(df_encoded.filter(regex='PART1_|PART2_').head())


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Inicializar o codificador
le_part1 = LabelEncoder()
le_part2 = LabelEncoder()

# Codificar as colunas
df_result02['PART1_encoded'] = le_part1.fit_transform(df_result02['PART1'])
df_result02['PART2_encoded'] = le_part2.fit_transform(df_result02['PART2'])

# Exibir as novas colunas
print(df_result02[['PART1', 'PART1_encoded', 'PART2', 'PART2_encoded']].head())


In [ ]:
# Exibir valores únicos em 'VALUE_ID'
print(df_result02['VALUE_ID'].unique())


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalização
scaler_minmax = MinMaxScaler()
df_result02['VALUE_ID_norm'] = scaler_minmax.fit_transform(df_result02[['VALUE_ID']])

# Padronização
scaler_standard = StandardScaler()
df_result02['VALUE_ID_std'] = scaler_standard.fit_transform(df_result02[['VALUE_ID']])

# Exibir as novas colunas
print(df_result02[['VALUE_ID', 'VALUE_ID_norm', 'VALUE_ID_std']].head())


In [ ]:
# Criar categorias usando pd.cut
df_result02['VALUE_ID_cat'] = pd.cut(df_result02['VALUE_ID'], bins=4, labels=False)

# Exibir a nova coluna
print(df_result02[['VALUE_ID', 'VALUE_ID_cat']].head())


In [ ]:
# Criar uma nova coluna combinando 'NAME' e 'VALUE_ID'
df_result02['NAME_VALUEID'] = df_result02['NAME'] + '_' + df_result02['VALUE_ID'].astype(str)

# Exibir a nova coluna
print(df_result02[['NAME', 'VALUE_ID', 'NAME_VALUEID']].head())


In [ ]:
# Label Encoding
le_name_valueid = LabelEncoder()
df_result02['NAME_VALUEID_encoded'] = le_name_valueid.fit_transform(df_result02['NAME_VALUEID'])

# Exibir a nova coluna
print(df_result02[['NAME_VALUEID', 'NAME_VALUEID_encoded']].head())


In [ ]:
# Converter 'DATA' para datetime
df_result02['DATA'] = pd.to_datetime(df_result02['DATA'], errors='coerce')

# Exibir a coluna convertida
print(df_result02['DATA'].head())


In [ ]:
# Extrair ano, mês, dia, hora, dia da semana
df_result02['ANO'] = df_result02['DATA'].dt.year
df_result02['MES'] = df_result02['DATA'].dt.month
df_result02['DIA'] = df_result02['DATA'].dt.day
df_result02['HORA'] = df_result02['DATA'].dt.hour
df_result02['DIA_SEMANA'] = df_result02['DATA'].dt.dayofweek  # 0 = segunda-feira

# Exibir as novas colunas
print(df_result02[['DATA', 'ANO', 'MES', 'DIA', 'HORA', 'DIA_SEMANA']].head())


In [ ]:
# Contar frequência de cada 'NAME'
name_counts = df_result02['NAME'].value_counts()

# Mapear a frequência para o DataFrame
df_result02['NAME_freq'] = df_result02['NAME'].map(name_counts)

# Exibir a nova coluna
print(df_result02[['NAME', 'NAME_freq']].head())


In [ ]:
# Contar frequência de cada 'VALUE_ID'
valueid_counts = df_result02['VALUE_ID'].value_counts()

# Mapear a frequência para o DataFrame
df_result02['VALUE_ID_freq'] = df_result02['VALUE_ID'].map(valueid_counts)

# Exibir a nova coluna
print(df_result02[['VALUE_ID', 'VALUE_ID_freq']].head())


In [ ]:
# Definir um limiar (por exemplo, média de 'VALUE_ID')
limiar = df_result02['VALUE_ID'].mean()

# Criar a feature binária
df_result02['VALUE_ID_high'] = df_result02['VALUE_ID'].apply(lambda x: 1 if x > limiar else 0)

# Exibir a nova coluna
print(df_result02[['VALUE_ID', 'VALUE_ID_high']].head())


In [ ]:
# Calcular a média de 'VALUE_ID' para cada 'NAME'
name_target_mean = df_result02.groupby('NAME')['VALUE_ID'].mean()

# Mapear a média para o DataFrame
df_result02['NAME_target_mean'] = df_result02['NAME'].map(name_target_mean)

# Exibir a nova coluna
print(df_result02[['NAME', 'NAME_target_mean']].head())


In [ ]:
# Verificar valores nulos
print("Valores nulos por coluna:")
print(df_result02.isnull().sum())

# Opcional: Preencher valores nulos ou remover linhas com nulos
df_result02 = df_result02.dropna()  


In [ ]:
# Passo 1: Agrupar por 'NAME' e 'VALUE_ID' e contar ocorrências
name_valueid_counts = df_result02.groupby(['NAME', 'VALUE_ID']).size().reset_index(name='Counts')

# Passo 2: Identificar combinações repetidas
repeating_combinations = name_valueid_counts[name_valueid_counts['Counts'] > 1]

# Passo 3: Mesclar com informações de 'KNR'
repeating_combinations_knr = pd.merge(repeating_combinations, df_result02[['NAME', 'VALUE_ID', 'KNR']], on=['NAME', 'VALUE_ID'], how='left')

# Passo 4: Remover duplicatas
repeating_combinations_knr = repeating_combinations_knr.drop_duplicates(subset=['NAME', 'VALUE_ID', 'KNR'])

# Passo 5: Criar mapeamento com o 'KNR' mais frequente
most_frequent_knr = df_result02.groupby(['NAME', 'VALUE_ID'])['KNR'].agg(lambda x: x.value_counts().index[0]).reset_index()
name_valueid_to_knr = most_frequent_knr.set_index(['NAME', 'VALUE_ID'])['KNR'].to_dict()

# Passo 6: Adicionar nova coluna ao DataFrame
df_result02['KNR_associated'] = df_result02.apply(lambda row: name_valueid_to_knr.get((row['NAME'], row['VALUE_ID']), None), axis=1)

# Passo 7: Verificar os resultados
df_result02['KNR_match'] = df_result02['KNR'] == df_result02['KNR_associated']
discrepancies = df_result02[df_result02['KNR_match'] == False]

# Exibir o DataFrame final
print(df_result02[['NAME', 'VALUE_ID', 'KNR', 'KNR_associated', 'KNR_match']])


## RESULTADOS 06

## Nas células a seguir foram selecionadas as colunas necessarias para o desenvolvimento desse modelo e o tratamento para que não existam dados duplicados. Por fim, foi gerado um novo dataset utilizando parquet.

In [ ]:
df_result06 = pd.read_csv('RESULTADOS_06_2023_07_2023_full_teste.csv', low_memory=False, compression='gzip')
df_result06.head()

In [ ]:
# Dropar uma coluna e modificar o DataFrame existente
df_result06.drop(['Unnamed: 0', 'NAME', 'STATUS', 'UNIT', 'VALUE_ID', 'VALUE'], axis=1, inplace=True)
df_result06.head()

In [ ]:
df_result06 = df_result06.drop_duplicates()
df_result06

In [ ]:
df_result06.to_parquet('df_result06.parquet', index=False)
df_result06 = pd.read_parquet('df_result06.parquet')
df_result06

## RESULTADOS 08

## Nas células a seguir foram selecionadas as colunas necessarias para o desenvolvimento desse modelo e o tratamento para que não existam dados duplicados. Por fim, foi gerado um novo dataset utilizando parquet.

In [ ]:
df_result08 = pd.read_csv('RESULTADOS_08_2023_09_2023_full_teste.csv', low_memory=False, compression='gzip')
df_result08.head()

In [ ]:
# Dropar uma coluna e modificar o DataFrame existente
df_result08.drop(['Unnamed: 0', 'NAME', 'STATUS', 'UNIT', 'VALUE_ID', 'VALUE'], axis=1, inplace=True)
df_result08.head()

In [ ]:
df_result08 = df_result08.drop_duplicates()
df_result08

In [ ]:
df_result08.to_parquet('df_result08.parquet', index=False)
df_result08 = pd.read_parquet('df_result08.parquet')
df_result08

## RESULTADOS 10

## Nas células a seguir foram selecionadas as colunas necessarias para o desenvolvimento desse modelo e o tratamento para que não existam dados duplicados. Por fim, foi gerado um novo dataset utilizando parquet.

In [ ]:
df_result10 = pd.read_csv('RESULTADOS_10_2023_11_2023_full_teste.csv', low_memory=False, compression='gzip')
df_result10.head()

In [ ]:
# Dropar uma coluna e modificar o DataFrame existente
df_result10.drop(['Unnamed: 0', 'NAME', 'STATUS', 'UNIT', 'VALUE_ID', 'VALUE'], axis=1, inplace=True)
df_result10.head()

In [ ]:
df_result10 = df_result10.drop_duplicates()
df_result10

In [ ]:
df_result10.to_parquet('df_result10.parquet', index=False)
df_result10 = pd.read_parquet('df_result10.parquet')
df_result10

## MERGE DOS ARQUIVOS RESULT

Nas células a seguir, foi realizado a junção dos dados das tabelas de resultado em um único arquivo para uso do conjunto de dados na construção do novo modelo.

In [ ]:
import pandas as pd

# Lista com os nomes dos arquivos Parquet
arquivos = [
    'df_result02.parquet',
    'df_result04.parquet',
    'df_result06.parquet',
    'df_result08.parquet',
    'df_result10.parquet'
]

# Inicializar uma lista vazia para armazenar os DataFrames
dfs = []

# Ler cada arquivo Parquet em um DataFrame e adicionar à lista
for arquivo in arquivos:
    df = pd.read_parquet(arquivo)
    dfs.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_merged = pd.concat(dfs, ignore_index=True)

# Salvar o DataFrame resultante em um novo arquivo Parquet
df_merged.to_parquet('df_merged.parquet')

print("Merge concluído e salvo como 'df_merged.parquet'")

In [49]:
df_merged = pd.read_parquet('df_merged.parquet')

## MERGE DOS ARQUIVOS RESULT E FAALHAS

Nessa célula foi realizada a junção de todos os dados necessários para a construção do modelo da sprint 2.

In [ ]:
# # Fazer a combinação usando a coluna comum 'KNR'
# # Nesse caso, estamos apenas combinando a coluna 'HALLE' da tabela df_falhas_predict com df_result
df_combined = pd.merge(df_merged, df_falhas[['KNR', 'HALLE', 'FALHA']], on='KNR', how='inner')

# # Visualizar o DataFrame resultante
print(df_combined.head())

# # Se necessário, salvar o resultado em um novo arquivo CSV
df_combined.to_parquet("df_result_combined.parqut", index=False)

In [51]:
df_result_combined = pd.read_parquet('df_result_combined.parquet')

In [ ]:
df_result_combined.head()

## Feature Engineering

In [ ]:
df_result_combined.shape

Aqui foram realizadas diversas tentativas para padrodizar os dados da coluna 'DATA' 

In [ ]:
# Primeiro, tenta converter diretamente, assumindo que a maioria dos valores tenha milissegundos
df_result_combined['DATA_FORMATADA'] = pd.to_datetime(df_result_combined['DATA'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce')

# Agora, lida com aqueles que falharam (são NaT) e tenta converter sem milissegundos
df_result_combined['DATA_FORMATADA'].fillna(pd.to_datetime(df_result_combined['DATA'], format="%Y-%m-%d %H:%M:%S", errors='coerce'), inplace=True)

In [ ]:
df_result_combined.head()

In [ ]:
df_result_combined

In [ ]:
df_result_combined['HALLE'].unique()

In [ ]:

# Ordenar por KNR e DATA_FORMATADA
df_result_combined = df_result_combined.sort_values(by=['KNR', 'DATA_FORMATADA'])

# Calcular o tempo entre falhas consecutivas dentro do mesmo carro e estação (HALLE)
df_result_combined['TEMPO'] = df_result_combined.groupby(['KNR', 'HALLE'])['DATA_FORMATADA'].diff().fillna(pd.Timedelta(seconds=0))

# Somar o tempo gasto em cada estação (HALLE) por cada carro (KNR)
tempo_por_estacao = df_result_combined.groupby(['KNR', 'HALLE'])['TEMPO'].sum().reset_index()

# Criar um DataFrame pivotado para armazenar a soma de tempo em colunas separadas
resultado = tempo_por_estacao.pivot(index='KNR', columns='HALLE', values='TEMPO').fillna(pd.Timedelta(seconds=0))

# Renomear as colunas para o formato desejado (ajustando para HALLE correto)
resultado.columns = ['SomaTempo1' if col == 'ZP5A' else 'SomaTempo2' if col == 'ZP5B' else 'SomaTempo718' for col in resultado.columns]

# Resetar o índice para transformar KNR em coluna
resultado = resultado.reset_index()

# Exibir o resultado final
print(resultado)


In [ ]:
resultado.columns

In [ ]:
resultado